In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
collatzs = []

In [3]:
n_iter = 100000    # number of seeds for collatz algorithm
#Create algorithm
for i in range(2,n_iter):
    c = i
    col = [c]
    while c > 1:
        if c%2==1:
            c = int(c * 3 + 1)
        else:
            c =int(c/ 2)
        col.append(c)
    collatzs.append(col)

In [4]:
#Choose parameters. Drop below a trunc_len and truncate the rest to it
trunc_len = 200

In [5]:
#Dropping too small branches
truncated = []
for i in range(len(collatzs)):
    if len(collatzs[i]) >= trunc_len:
        truncated.append(collatzs[i])

In [6]:
#Truncating branches
for i in range(len(truncated)):
    truncated[i] = truncated[i][-trunc_len:]

In [8]:
#Prune duplicate branches
trunc_pruned = []
end_values = []
for branch in truncated:
    if branch[0] not in end_values:
        trunc_pruned.append(branch)
        end_values.append(branch[0])
    else:
        pass
len(trunc_pruned), len(truncated), len(collatzs)

(562, 4469, 99998)

<i>We are left with very few values compared to our origin (and computing time)

Converting to numpy

In [9]:
collatz = np.row_stack(trunc_pruned)

In [10]:
collatz = np.flip(collatz, axis=1)

In [11]:
#Create increment - -8 degrees if odd, + 8 degrees if even
increment = np.zeros(collatz.shape)
increment[collatz%2==1] = -2 * np.pi / 45
increment[collatz%2==0] = 2 * np.pi / 45
increment[collatz==1] = 0

In [12]:
# Cumulative sum of increment to find full direction
alpha = increment.cumsum(axis=1)

In [13]:
#Initialize d for distance and theta for angle. Will convert to Cartesian coordinates in tableau
d = np.ones(collatz.shape)
theta = np.zeros(collatz.shape)

In [14]:
# Parameters for creating distance and theta algorithm
n_branches = collatz.shape[0]
steps_per_branch = collatz.shape[1]

In [15]:
#Trigonometry algorithm. Described in blog post ___
for i in range(1, steps_per_branch):
    beta = np.zeros(n_branches)
    beta[increment[:,i] < 0] = alpha[increment[:,i]<0][:,i] + np.pi - theta[increment[:,i]<0][:,i-1]
    beta[increment[:,i] > 0] = np.pi - alpha[increment[:,i]>0][:,i] + theta[increment[:,i]>0][:,i-1]
    
    d[:,i] = np.sqrt(d[:,i-1]**2 + 1 - 2 * d[:,i-1] * np.cos(beta))
    
    mu = np.arcsin(np.sin(beta) / d[:,i])
    mu[increment[:,i] < 0] = -mu[increment[:,i] < 0]
    
    theta[:,i] = theta[:,i-1] + mu

In [16]:
theta

array([[0.        , 0.06981317, 0.13962634, ..., 6.12329168, 6.14744629,
        6.17122146],
       [0.        , 0.06981317, 0.13962634, ..., 6.12244366, 6.14511968,
        6.1692051 ],
       [0.        , 0.06981317, 0.13962634, ..., 1.91192168, 1.89631122,
        1.88737298],
       ...,
       [0.        , 0.06981317, 0.13962634, ..., 6.20849306, 6.23126461,
        6.2565327 ],
       [0.        , 0.06981317, 0.13962634, ..., 6.22715531, 6.25446958,
        6.28475901],
       [0.        , 0.06981317, 0.13962634, ..., 1.68629417, 1.69686121,
        1.71018003]])

In [17]:
d

array([[ 1.        ,  1.9951281 ,  2.98053614, ..., 41.19368828,
        41.08101008, 40.85440796],
       [ 1.        ,  1.9951281 ,  2.98053614, ..., 40.91752809,
        40.53384953, 40.30513237],
       [ 1.        ,  1.9951281 ,  2.98053614, ..., 22.59273246,
        23.52572801, 24.5024304 ],
       ...,
       [ 1.        ,  1.9951281 ,  2.98053614, ..., 36.8434321 ,
        36.2896094 , 35.8788425 ],
       [ 1.        ,  1.9951281 ,  2.98053614, ..., 31.17119641,
        30.6349102 , 30.24771126],
       [ 1.        ,  1.9951281 ,  2.98053614, ..., 36.63778292,
        37.55775595, 38.42032886]])

In [18]:
#Need to flatten the rows, generate ids, generate index

In [19]:
group_index = np.resize(np.arange(steps_per_branch),(collatz.shape))

In [20]:
group_id = np.repeat(np.arange(n_branches),steps_per_branch)

In [21]:
df = pd.DataFrame(collatz.ravel(), columns=['collatz'])

In [22]:
df['theta'] = theta.ravel()
df['d'] = d.ravel()
df['group_id'] = group_id
df['group_index'] = group_index.ravel()

In [23]:
collatz.shape

(562, 200)

In [24]:
df

,collatz,theta,d,group_id,group_index
0,1,0.000000,1.000000,0,0
1,2,0.069813,1.995128,0,1
2,4,0.139626,2.980536,0,2
3,8,0.209440,3.951423,0,3
4,16,0.279253,4.903060,0,4
...,...,...,...,...,...
112395,608188,1.659982,34.878269,561,195
112396,1216376,1.671743,35.787862,561,196
112397,2432752,1.686294,36.637783,561,197
112398,810917,1.696861,37.557756,561,198


In [25]:
df.to_csv('collatz_tree.csv')